![CH8-ADS.png](.\Media\CH8-ADS.png)
# <span style="color:#cc5500;">Deploy Azure SQL VM from PowerShell </span>

In this Notebook, we will eplore you you can deploy and Azure SQL VM from PowerShell.

Why is this covered?  Most people when the deploy Azure Services for the first time, use the Azure Portal just like we did in the previous notebook.  The Portal is great for learning how SQL Server is installed to an Azure VM with the many menu-driven options and drop-down selections.  Using the Portal is likely the best option if you just need to get something deployed and won't have to do it more than once or twice.  However, when using the Portal, it is very easy to deploy your SQL Servers 'differently'.  For example, maybe the first time you deployed SQL, you chose a SQL Server 2016 Enterprise Edition with HDD disks on Windows Server 2016.  Then, you deploy a 2nd SQL Server, but you didn't realize that you chose a different version of SQL and you used Premium SSD disks.  Generally in an enterprise, you want all things to be consistent so that you can have good control of performance charasteristics.

This is the beauty of IaaC (Infrastructure as Code).  You create repeatable scripts, so that you have a consistent deployments over and over again.

Unlike the previous chapters in this notebook, you will not be using the SQL kernel in the code blocks.  Rather, in the cell blocks that follow make sure you have the PowerShell Kernel selected.

Azure Data Studio, by default, is pre-set to connect to our Commercial Azure Data Centers.  However, there are Sovereign Cloud data centers as well like Azure US Government.  I have seen very few youtube videos and on-line articles that discuss this.  As such, I will demonstrate the nuances of getting Azure Data Studio connected to a US Government Data Center where I happen to have a subscription.  If you only have a subscription in Azure Commercial, this script will work, you will just need to make the appropriate adustments to the code below when I prompt you to.

First go to the Settings Gear on the lower Left and select Settings \> Extensions \> Azure Account Configuration and make sure you have the desired "Soverign" Cloud checked if you have a subscription there.  If not, you can just use the default option.

![CH8-AzureAccount.jpg](.\Media\CH8-AzureAccount.jpg)


### <span style="color:rgb(0, 204, 153);">Run the Code block below</span>

1. Each Code block below is meant to be run individualy, one at a time
2. When you run the first block below, you will be prompted for your credentials
3. Ensure you have the PowerShell Kernel selected
4. View the results of the query by scrolling down to the results set

In [ ]:
# Make Initial Connection.  Azure Data Studio connects to Azure Commercial data centers.
# If connecting to Azure Gov, it must be specified in the -EnviornmentName, like my sample below, otherwise remove that and just use Connect-AzAccount
# When you run this command, have your account ready and your password ready.  It will pop open a window for you to enter your credentials

Connect-AzAccount -EnvironmentName AzureUSGovernment

In [ ]:
# It just so happens that I have multiple subscriptions, I need to display a list of my subscriptions so that I can pick the one I want to use
Get-AzSubscription

In [ ]:
# If you forget to set the scope (or context) of the Azure PowerSell Az commands to the correct Azure Subscription, then you may end up provisioning or deleting resources in the wrong Azure Subscription
# From the results from the above command, enter the Subscription name below in this block.

Set-AzContext -SubscriptionName "US Gov FedCon"

In [ ]:
# Get a list of Azure Locations so you know which location to add to your global variable

Get-AzLocation

In [ ]:
# Set Global variables.  I am going to choose one of my locations which is US Gov Virginia
# Enter your choice for $Location, and choose a lower-case name for your Resource Group Name 

$Location = "usgovvirginia"
$ResourceGroupName = "rgazsqlvm"


In [ ]:
# Storage.  I am using Premium LRS disks, but you can choose other types
$StorageName = $ResourceGroupName + "storage"
$StorageSku = "Premium_LRS"

In [ ]:
# Network Properties.  You can use your own IP Address space.  I'm keeping my simple for this example

$InterfaceName = $ResourceGroupName + "ServerInterface"
$NsgName = $ResourceGroupName + "nsg"
$VNetName = $ResourceGroupName + "VNet"
$SubnetName = "Default"
$VNetAddressPrefix = "10.0.0.0/16"
$VNetSubnetAddressPrefix = "10.0.0.0/24"
$TCPIPAllocationMethod = "Dynamic"
$DomainName = $ResourceGroupName

In [ ]:
# Compute.  I am choosing the Standard DS1 V2 VM.  Check which VM's are available in your Region as this size may not be available for you.  The VM's do tend to vary from 
# region to region.

$VMName = $ResourceGroupName + "VM"
$ComputerName = $ResourceGroupName + "Server"
$VMSize = "Standard_DS1_V2"
$OSDiskName = $VMName + "OSDisk"

In [ ]:
# Get a list of the Gallery Images available in this region so that you can make the proper choice in the next code block.  
Get-AzVMImageOffer -Location $Location -Publisher 'MicrosoftSQLServer'

In [ ]:
# Image.  I ma choosing a SQL Server Gallery image that is SQL Server 2017 on a Windows Server 2016 Image.  I choose the Developers Edition.
#  We have many gallery images to choose from.

$PublisherName = "MicrosoftSQLServer"
$OfferName = "SQL2017-WS2016"
$Sku = "SQLDEV"
$Version = "latest"


In [ ]:
# Okay, here is where we run all the commands to create a SQL Server on Azure VM
# This script will take a while to run
# Watch for the prompt to enter a user account and password.  I choose User: sqladmin  Password: <enter-a-hardpasswordname>
# Review the results at the bottom with the script completes
# Log into the Azure Portal to see your resources and to connect to your SQL Server

# Resource Group
New-AzResourceGroup -Name $ResourceGroupName -Location $Location

# Storage
$StorageAccount = New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name $StorageName -SkuName $StorageSku -Kind "Storage" -Location $Location

# Network
$SubnetConfig = New-AzVirtualNetworkSubnetConfig -Name $SubnetName -AddressPrefix $VNetSubnetAddressPrefix
$VNet = New-AzVirtualNetwork -Name $VNetName -ResourceGroupName $ResourceGroupName -Location $Location -AddressPrefix $VNetAddressPrefix -Subnet $SubnetConfig
$PublicIp = New-AzPublicIpAddress -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -AllocationMethod $TCPIPAllocationMethod -DomainNameLabel $DomainName
$NsgRuleRDP = New-AzNetworkSecurityRuleConfig -Name "RDPRule" -Protocol Tcp -Direction Inbound -Priority 1000 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 3389 -Access Allow
$NsgRuleSQL = New-AzNetworkSecurityRuleConfig -Name "MSSQLRule"  -Protocol Tcp -Direction Inbound -Priority 1001 -SourceAddressPrefix * -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 1433 -Access Allow
$Nsg = New-AzNetworkSecurityGroup -ResourceGroupName $ResourceGroupName -Location $Location -Name $NsgName -SecurityRules $NsgRuleRDP,$NsgRuleSQL
$Interface = New-AzNetworkInterface -Name $InterfaceName -ResourceGroupName $ResourceGroupName -Location $Location -SubnetId $VNet.Subnets[0].Id -PublicIpAddressId $PublicIp.Id -NetworkSecurityGroupId $Nsg.Id

# Compute
$VirtualMachine = New-AzVMConfig -VMName $VMName -VMSize $VMSize
$Credential = Get-Credential -Message "Type the name and password of the local administrator account."
$VirtualMachine = Set-AzVMOperatingSystem -VM $VirtualMachine -Windows -ComputerName $ComputerName -Credential $Credential -ProvisionVMAgent -EnableAutoUpdate #-TimeZone = $TimeZone
$VirtualMachine = Add-AzVMNetworkInterface -VM $VirtualMachine -Id $Interface.Id
$OSDiskUri = $StorageAccount.PrimaryEndpoints.Blob.ToString() + "vhds/" + $OSDiskName + ".vhd"
$VirtualMachine = Set-AzVMOSDisk -VM $VirtualMachine -Name $OSDiskName -VhdUri $OSDiskUri -Caching ReadOnly -CreateOption FromImage

# Image
$VirtualMachine = Set-AzVMSourceImage -VM $VirtualMachine -PublisherName $PublisherName -Offer $OfferName -Skus $Sku -Version $Version

# Create the VM in Azure
New-AzVM -ResourceGroupName $ResourceGroupName -Location $Location -VM $VirtualMachine

# Add the SQL IaaS Extension, and choose the license type
New-AzSqlVM -ResourceGroupName $ResourceGroupName -Name $VMName -Location $Location -LicenseType PAYG